# Feature Engineering


## Case: Lending Club

Mais um conjunto de dados do Kaggle. O desafio pode ser visto [aqui](https://www.kaggle.com/wendykan/lending-club-loan-data). Lá, estão disponíveis várias das soluções (Kernels) realizadas para esse desafio. 

O Lending Club (ou Clube de empréstimos, em tradução livre) é uma empresa de empréstimo peer-to-peer com sede nos Estados Unidos, na qual os investidores fornecem fundos para potenciais tomadores de empréstimos e os investidores obtêm um lucro, dependendo do risco que correm (a pontuação de crédito do tomador). O Lending Club faz a "ponte" entre investidores e quem precisa de empréstimo.

<img src="../images/loan.png" width = 600/>


Os dados que vamos trabalhar neste notebook contém dados completos de empréstimos realizados entre 2007-2015.

## Download dos dados

O arquivo de dados original (que está lá no Kaggle), tem 1.2Gb. Fiz um filtro para usarmos somente as linhas referentes a empréstimos que já foram liquidados:

In [ ]:
#df_complete = pd.read_csv('loan_complete.csv')
#df = df_complete[(df_complete.loan_status != 'Current')]
#df.to_csv(loan.csv)

Ainda assim, o arquivo tem 800Mb (e 329 se comprimirmos em um zip) e o GitHub só aceita arquivos de até 50M, pois o objetivo central do Git é armazenar o histórico de CÓDIGOS! 

O tamanho dos dados ser superior a esse limite é extremamente comum e por isso, daqui para frente vou manter dados no Google Drive e inserir na aula o download deles.

O arquivo dos dados que trabalharemos neste notebook tem 329Mb e com essa biblioteca do Google Drive `google_drive_downloader`, conseguimos acompanhar o progresso do download com o parâmetro `showsize = True`.

<!--- !pip install googledrivedownloader ---> 

In [ ]:
import os
import zipfile
from google_drive_downloader import GoogleDriveDownloader as gdd

def download_dataset(dataset_path, google_id):
    if not os.path.exists(dataset_path):
        gdd.download_file_from_google_drive(file_id=google_id,
                                            dest_path=dataset_path, 
                                            showsize = True)
        ''
def unzip_dataset(dataset_path, unzipped_dataset_path):
    if not os.path.exists(unzipped_dataset_path):
        print("Unzipping ...")
        with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
            zip_ref.extractall('../data/')
        print("Done!")

dataset_path = '../data/loan.zip'
google_id = '1GrXYO6WjPAMwb1czTtgsdwU0_VtYBuKQ'
download_dataset(dataset_path, google_id)

unzipped_dataset_path = '../data/loan.csv'
unzip_dataset(dataset_path, unzipped_dataset_path)

## Antes de começarmos: análise exploratória

A análise exploratória, pelo menos uma básica, temos que fazer SEMPRE que nos depararmos com um novo conjunto de dados. Vamos novamente usar as operações que aprendemos para entender um pouco do contexto desses dados, principalmente as colunas que queremos selecionar e as características que queremos/podemos extrair.

In [ ]:
import numpy as np
import pandas as pd

#maximo de colunas
pd.set_option("display.max_rows",150)

In [ ]:
df = pd.read_csv('../data/loan.csv', nrows = 200000, low_memory=False)
df.head()

Vamos dar uma olhada na lista das colunas:

Temos mais de 1 milhão de tomadores de empréstimos nessa base, ou seja, cada linha (instância) temos as características (features) de uma pessoa que pediu empréstimo:

In [ ]:
len(df)

In [ ]:
list(df.columns)

<div class="alert alert-block alert-info">
<b>Exercício:</b>
Carreguem o arquivo "loan_metadata.xlsx" e explorem suas colunas e os significados delas. 
Selecionem duas listas de colunas que julgarem serem úteis no cenário de previsão de um empréstimo ser pago ou não: a primeira com colunas numéricas e a segunda com colunas não-numéricas.</div>

## Features numéricas
Num primeiro momento, vamos selecionar do nosso conjunto algumas *features* (colunas) numéricas, pois comumente, modelos de *machine learning* só aceitam esse tipo de dado.

In [ ]:
initial_vars = ['loan_amnt', 'int_rate', 'installment', 'annual_inc',
       'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc']

In [ ]:
numeric_df = df[initial_vars]

Se a feature é numérica, posso simplesmente usar? 

- Exemplo:

<img src="../images/exe.png" alt="exemplo" width="500"/>


### Padronização e normalização

![standardization](../images/standardization.jpg)

- Padronização ou regularização: uma adequação bastante usada quando lidamos com features numéricas é mapear os valores de uma distribuição para valores de uma distribuição normal padrão para que, independentemente dos valores que temos na distruição, tenhamos a mesma **grandeza** de valores.
- Distribuição normal padrão: uma distribuição normal que tem média 0 e desvio padrão 1.
- Cálculo: para realizar esse mapeamento, calcula-se a diferença entre cada um dos valores e a média e dividimos essa diferença pelo desvio padrão.

<img src="../images/standardization-calc.jpg" alt="calculo_padronizacao" width="400"/>

- Exemplo: 

![standardization_exe](../images/standardization-exe.jpg)

- **Normalização**: tem como objetivo colocar os valores da distribuição dentro do intervalo de 0 e 1 ou, caso tenha resultado negativo, entre -1 e 1.

<img src="../images/norm-calc.png" alt="calculo_norm" width="200"/>


#### Atenção!!!
Se a distribuição não é normal (Gaussiana) ou se o desvio padrão é muito pequeno, a melhor escolha é **normalizar**.

#### Padronização/normalização no Python e o Scikit-learn

O pacote [Scikit-learn](https://scikit-learn.org/stable/) (`sklearn`) é uma biblioteca Python para Machine Learning.
Vamos usar **bastante** essa biblioteca. Ela tem muita coisa de ML pronta (e bem feita) e uma documentação exemplar.

In [ ]:
import sklearn

Voltando aos nossos dados numéricos, podemos usar a padronização do Scikit-learn...

Mas antes, **precisamos substituir os nulos**:

In [ ]:
#verificando a porcentagem de nulos por coluna
numeric_df.isnull().mean().sort_values(ascending=False).head(50)

Podemos fazer a substituição pela média ou mediana, por exemplo.

\* Uma forma um pouco mais robusta: média condicionada (selecionar grupos de dados com base em alguma característica e substituir pela média dentro do grupo)

<div class="alert alert-block alert-info">
<b>Exercício:</b>
Calcule a média e a mediana das colunas numéricas selecionadas.
</div>

<div class="alert alert-block alert-info">
<b>Exercício:</b>
Preencha os valores nulos das colunas com a média ou a mediana.
</div>

Agora sim, podemos aplicar a padronização usando o [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) do Scikit-learn:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

In [ ]:
numeric_df.head(3)

In [ ]:
#os scalers do Scikit-learn retornam listas numpy como resposta
#se quisermos acessar esses dados em forma de Dataframe, precisamos fazer a conversão:
numeric_df_scaled = pd.DataFrame(scaled_data, columns = numeric_df.columns)
numeric_df_scaled.head(3)

Existem outros padronizadores/normalizadores, como o [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html), com o qual conseguimos fazer a normalização comentada acima.

## Features categóricas

Em vários algoritmos e modelos de IA, só são aceitos valores numéricos. Então precisamos de uma forma de mapear (*encoders*) as categorias para números.

In [ ]:
cat_cols = ['term', 'grade', 'emp_length', 'home_ownership', 'application_type', 
            'verification_status', 'pymnt_plan', 'purpose']

### Label Encoder

Cada valor único da feature categórica é mapeado para um ID único. No Scikit-learn: [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

<div class="alert alert-block alert-info">
<b>Exercício:</b>
Com a documentação do LabelEncoder do Scikit-learn, aplique a função "fit_transform" na coluna categórica "term".
</div>

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#preencha aqui

Também é necessário preencher os nulos para que esse *encoder* funcione, mas podemos preencher com uma string vazia para que todas as linhas para as quais essa informação não existe receba um mesmo ID. Vamos aplicar o encoder para todas as colunas categóricas e já preencher os nulos.

In [ ]:
new_df = pd.DataFrame()
for col in cat_cols:
    new_df[col] = df[col].fillna('')
    new_df[col] = None

select_df = pd.concat([numeric_df, new_df], axis = 1)

In [ ]:
select_df[cat_cols].head()

**Atenção:** 
- Os IDs são numéricos e, portanto, **implicam uma ordem**! 
- Por isso, só pode ser usado com algoritmos não lineares baseados em árvore.

### One hot encoding

- Usado com a maioria dos algoritmos lineares
- Podemos excluir a primeira coluna evita colinearidade
- Não introduz ordem ao atributo
- Aumenta a **dimensionalidade** do dataset

In [ ]:
select_df[cat_cols].max()

Vamos usar o [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) do Scikit-learn.

In [ ]:
new_df = pd.DataFrame()
for col in cat_cols: 
    col_dummies = pd.get_dummies(df[col], drop_first=True, prefix = col)
    new_df = pd.concat([new_df, col_dummies], axis = 1)

select_df = pd.concat([numeric_df, new_df], axis = 1)

In [ ]:
select_df.head()

### Outros enconders

#### Effect Coding
- Bem similar ao *dummy*;
- Substitui por -1 todos os 0s que representam uma das categorias (primeira ou última) no dummy encoding;
- Exemplo nesse [link](https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63).

- [Outro exemplo](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faqwhat-is-effect-coding/).

#### Bin-counting 
- Maldição da dimensionalidade, categorias "grandes" (com muitos valores distintos);
- Usa probabilidade baseada em informação estatística da relação do valor com o target;
- Exemplo simplificado do [link 2](https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63): construir valores de probabilidade de um ataque DDoS ser causado por qualquer um dos endereços IP baseado em **dados históricos** de endereços IP relacionados a ataques DDoS.

#### Feature Hashing Scheme
- Maldição da dimensionalidade;
- **Função Hash** tipicamente usada com um número pré-definido de features resultantes (vetor de comprimento pré-definido) de forma que os valores de hash dos valores da categoria sejam usados como índices nesse vetor predefinido e os valores sejam atualizados de acordo.
- Funções Hash mapeiam uma grande quantidade de valores em uma quantidade finita (e pequena) de valores -> colisões.
- [Explicação básica sobre funções hash](https://www.techtudo.com.br/artigos/noticia/2012/07/o-que-e-hash.html)
- [Função Scikit Learn](https://scikit-learn.org/0.19/modules/generated/sklearn.feature_extraction.FeatureHasher.html)

## Outras práticas para CRIAÇÃO de features

- A partir de qualquer tipo de feature (numérica ou categórica), podemos criar outras features, tanto derivando informações (ex: a partir da taxa de juros anual derivar a taxa mensal) quanto enriquecendo o conjunto de dados (ex: cruzando as informações que eu tenho no meu conjunto de dados para trazer novas informações de outro conjunto).

- Um exemplo é quando há valores muito raros e próximos de valores mais numerosos, pois podemos agrupar esses valores por ter maior interesse em um grupo - ex: público alvo por idade.

- Outro exemplo bastante comum é usar **dados geográficos** para criar features. Um exemplo no nosso case: mapear a coluna do estado (`add_state`) para uma coluna de região (que dei o nome de `region`):

In [ ]:
df['addr_state'].unique()

west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']

df['region'] = np.nan

def finding_regions(state):
    ...

- Enriquecimento de dados geográficos: podemos agregar mais informação a partir dos dados geográficos que temos no conjunto de dados. [Nesse link](https://medium.com/creditas-tech/incrementando-dados-geogr%C3%A1ficos-com-o-censo-nacional-do-ibge-54d342c4bdcf) é exemplificado essa técnica com dados do IBGE.

## Material para aprofundamento:
Para mais detalhes sobre outros métodos de Feature Engineering veja [esses slides](https://www.dropbox.com/s/mebc9i6kxgm516c/feature-engineering-mlmeetup.pdf?dl=0) do HJ van Veen